# Avocado Prices Data Analysis

This notebook explores historical avocado price data across multiple U.S. regions and avocado types.  
The goal of this analysis is to identify trends, seasonal patterns, and differences between organic and conventional avocados using structured datasets provided in multiple formats (CSV, JSON, Excel, SQLite).

This project follows the data science lifecycle, including:

- Data loading and inspection
- Data cleaning and preparation
- Exploratory Data Analysis (EDA)
- Visualization and interpretation
- Cross-format data integration
- Validation using SQL


## Setup and Library Imports

The analysis begins by importing the required Python libraries. 


In [ ]:
!pip install openpyxl


In [ ]:
!pip install seaborn


In [ ]:
!pip install pandas numpy matplotlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")


## Loading the Primary Dataset (CSV)

The primary avocado dataset is loaded from a CSV file obtained from Kaggle.  
This dataset contains historical price and sales volume information across multiple regions and avocado types.

Initial inspection steps are performed to understand the dataset structure, including:

- Viewing sample rows
- Checking dataset dimensions
- Examining data types


In [ ]:
df = pd.read_csv("avocado.csv")
df.head()
df.shape
df.info()


## Initial Exploratory Data Analysis (EDA)

Before cleaning the data, exploratory analysis is performed to understand the dataset’s characteristics.

Key checks include:

- Summary statistics
- Missing values
- Unique categories (type and region)

This step helps identify necessary cleaning operations and guides later analysis decisions.


In [ ]:
df.describe()
df.isna().sum()
df['type'].unique()
df['region'].unique()


## Data Cleaning and Preparation

Several preprocessing steps are required to ensure the dataset is suitable for analysis:

- The `Date` column is converted to a datetime format for time-based analysis.
- Numeric fields such as `AveragePrice` and `Total Volume` are validated.
- Missing or inconsistent values are handled.
- New features (Year and Month) are created to enable seasonal analysis.

These transformations improve data quality and analytical flexibility.


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])


In [ ]:
df['AveragePrice'] = pd.to_numeric(df['AveragePrice'], errors='coerce')
df['Total Volume'] = pd.to_numeric(df['Total Volume'], errors='coerce')


In [ ]:
df = df.dropna()


In [ ]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month


## Data Visualization and Trend Analysis

Visualizations are used to reveal patterns that are difficult to detect from raw tables alone.

The following aspects are explored:

- Price trends over time
- Organic vs. conventional comparisons
- Regional price differences
- Seasonal price behavior

Each visualization is followed by interpretation to connect graphical patterns with meaningful insights.


In [ ]:
plt.figure()
df.groupby('Date')['AveragePrice'].mean().plot()
plt.title("Average Avocado Price Over Time")
plt.ylabel("Price")
plt.show()


In [ ]:
plt.figure()
sns.boxplot(data=df, x='type', y='AveragePrice')
plt.title("Price Distribution by Avocado Type")
plt.show()


In [ ]:
plt.figure()
sns.barplot(data=df, x='region', y='AveragePrice')
plt.xticks(rotation=90)
plt.title("Average Price by Region")
plt.show()


In [ ]:
plt.figure()
sns.lineplot(data=df, x='Month', y='AveragePrice')
plt.title("Seasonal Price Trends")
plt.show()


## Loading Secondary Dataset (JSON – New York)

A secondary dataset containing New York observations is loaded from a JSON file.

This file uses JSON Lines format and stores dates as Unix timestamps in milliseconds.  
To correctly interpret the data:

- JSON Lines parsing is enabled
- Automatic date conversion is disabled
- Dates are manually converted using Pandas

This step highlights real-world challenges associated with this data format. 


In [ ]:
with open("avocado_secondary_NY.json", "r") as f:
    print(f.read()[:500])


In [ ]:
ny_df = pd.read_json(
    "avocado_secondary_NY.json",
    lines=True,
    convert_dates=False
)


In [ ]:
ny_df['Date'] = pd.to_datetime(ny_df['Date'], unit='ms')


In [ ]:
ny_df = pd.read_json("avocado_secondary_NY.json", lines=True)


In [ ]:
ny_df = pd.read_json(
    "avocado_secondary_NY.json",
    lines=True,
    convert_dates=False
)

ny_df['Date'] = pd.to_datetime(ny_df['Date'], unit='ms')

ny_df.head()
ny_df.info()


## Loading Secondary Dataset (Excel – San Francisco)

San Francisco data is provided in Excel format.  
Reading Excel files in Pandas requires the optional dependency `openpyxl`, which serves as the engine for `.xlsx` files.

After loading, the dataset is inspected to verify:

- Column structure
- Data types
- Compatibility with other datasets


In [ ]:
sf_df = pd.read_excel("avocado_secondary_SF.xlsx")
sf_df.head()
sf_df.info()


In [ ]:
ny_df['Date'] = pd.to_datetime(ny_df['Date'])
sf_df['Date'] = pd.to_datetime(sf_df['Date'])


In [ ]:
ny_df.columns
sf_df.columns


In [ ]:
sf_df.rename(columns={'Average Price': 'AveragePrice'}, inplace=True)


## Merging JSON and Excel Datasets

The New York and San Francisco datasets are combined to reconstruct a multi-region dataset.

Before merging:

- Column names are aligned
- Data types are standardized
- Date formats are verified



In [ ]:
merged_secondary = pd.concat([ny_df, sf_df], ignore_index=True)
merged_secondary.shape


In [ ]:
merged_secondary.describe()
df.describe()


## Validation Using SQLite Database

To verify dataset consistency, the merged dataset is compared against the SQLite database version.

Validation checks include:

- Column alignment
- Data type comparisons
- Row counts
- Summary statistics

This process ensures that transformations and merges preserved data integrity.


In [ ]:
conn = sqlite3.connect("avocado_secondary_ALL.db")

sql_df = pd.read_sql_query("SELECT * FROM avocado_all", conn)
sql_df.info()
sql_df.shape


In [ ]:
set(sql_df.columns) - set(merged_secondary.columns)


## Exporting Processed Data

A filtered subset of the cleaned dataset is exported to a new structured format.

Exporting data is a common step in real-world workflows, enabling:

- Data sharing
- Reproducibility
- Downstream analysis


In [ ]:
organic_df = df[df['type'] == 'organic']
organic_df.to_csv("organic_avocados.csv", index=False)


## Conclusion

This analysis demonstrated the full data science workflow, including data cleaning, visualization, cross-format integration, and validation.

Key insights include:

- Observable price fluctuations over time
- Differences between avocado types
- Regional variation
- Seasonal effects

The project also reinforced the importance of handling file formats, data types, and reproducible workflows.
